In [1]:
from datetime import datetime
import os
import pickle
from pandas import Series, DataFrame
import pandas as pd

### requirement
# tally
# skewer
# seqtk
# bowtie

In [2]:
fastq_path = '^fastq/'
filepath = '^resuts/'
files = ['ks175']

In [3]:
tally_option = '--nozip --with-quality' # will give unzipped file (will uncompile)

# for skewer
skewer_input_extension = '-unique'
skewer_option = '-Q 10'
skewer_adapter = 'NNNNNNCACTCGGGCACCAAGGA'

# for seqtk trimfq
fivelength_mate1 = '4'
threelength_mate1 = '0'
seqtk_input_extensition = '-unique-trimmed.fastq'
seqtk_output_extensition = '-unique-trimmed-tk.fastq'


# for bowtie
inpath = fastq_path
outpath = filepath
bowtie_input_extension = seqtk_output_extensition

# for mapping
mapping = '_3map'

# for RPKM
gene_type = '_gene'

In [4]:
# tally for UMI "http://wwwdev.ebi.ac.uk/enright-dev/kraken/reaper/src/reaper-latest/doc/tally.html"
tally = '^tools/tally/tally'

for f in files:
    inputfile = fastq_path+f+'.fastq.gz'
    outputfile = fastq_path+f+'-unique.fastq'
    
    command = '%s -i %s -o %s %s' % (
        tally,
        inputfile,
        outputfile,
        tally_option
        )

    print command
    os.system(command)

^tools/tally/tally -i ^fastq/ks175.fastq.gz -o ^fastq/ks175-unique.fastq --nozip --with-quality


In [5]:
# skewer
skewer = '^tools/skewer022/skewer-0.2.2-linux-x86_64'

log_file = filepath+'log_skewer.txt'
###
present = str(datetime.now())
if not os.path.isfile(log_file):
    logs = open(log_file, 'w')
    logs.write(present+'\n')
    logs.close()
else:
    logs = open(log_file, 'a')
    logs.write('\n'+present+'\n')
    logs.close()     
###

for f in files:
    file_path = fastq_path+f+skewer_input_extension+'.fastq'
    
    command = '%s %s -x %s %s >> %s' % (
        skewer,
        skewer_option,
        skewer_adapter,
        file_path,
        log_file
        ) 
    
    logs = open(log_file, 'a+')
    logs.write('\n\n'+f+'\n')
    logs.close()
    
    os.system(command)
    
print 
for line in open(log_file, 'r'):
    print line[:-1]


2019-08-01 15:43:51.352227


ks175
.--. .-.
: .--': :.-.
`. `. : `'.' .--. .-..-..-. .--. .--.
_`, :: . `.' '_.': `; `; :' '_.': ..'
`.__.':_;:_;`.__.'`.__.__.'`.__.':_;
skewer v0.2.2 [April 4, 2016]
Parameters used:
-- 3' end adapter sequence (-x):	NNNNNNCACTCGGGCACCAAGGA
-- maximum error ratio allowed (-r):	0.100
-- maximum indel error ratio allowed (-d):	0.030
-- mean quality threshold (-Q):		10
-- minimum read length allowed after trimming (-l):	18
-- file format (-f):		Sanger/Illumina 1.8+ FASTQ (auto detected)
-- minimum overlap length for adapter detection (-k):	3
Thu Aug  1 15:43:51 2019 >> started

Thu Aug  1 15:45:07 2019 >> done (75.537s)
23714432 reads processed; of these:
   27493 ( 0.12%) reads filtered out by quality control
  398874 ( 1.68%) short reads filtered out after trimming by size control
    4035 ( 0.02%) empty reads filtered out after trimming by size control
23284030 (98.19%) reads available; of these:
23284030 (100.00%) trimmed reads available after process

In [6]:
for f in files:
    inputfile_mate1 = fastq_path+f+seqtk_input_extensition
    outputfile_mate1 = fastq_path+f+seqtk_output_extensition

    command_mate1 = 'seqtk trimfq -b %s -e %s %s > %s'%(
        fivelength_mate1,
        threelength_mate1,
        inputfile_mate1,
        outputfile_mate1
        )    

    print command_mate1
    os.system(command_mate1)

seqtk trimfq -b 4 -e 0 ^fastq/ks175-unique-trimmed.fastq > ^fastq/ks175-unique-trimmed-tk.fastq


In [7]:
log_file = outpath+'log_bowtie.txt'
###
present = str(datetime.now())
if not os.path.isfile(log_file):
    logs = open(log_file, 'w')
    logs.write(present+'\n')
    logs.close()
else:
    logs = open(log_file, 'a')
    logs.write('\n'+present+'\n')
    logs.close()    
###


path_l = outpath + 'alignments/ladder/'
path_t = outpath + 'alignments/tRNA/'
path_r = outpath + 'alignments/rRNA/'
path_s = outpath + 'alignments/ssrAssrS/'
path_lf = outpath + 'alignments/lacI_ffs/'
path_chr = outpath + 'alignments/chr/'
path_temp = outpath + 'tmpds/'

if not os.path.exists(path_l):
    os.makedirs(path_l)
if not os.path.exists(path_t):
    os.makedirs(path_t)
if not os.path.exists(path_r):
    os.makedirs(path_r)
if not os.path.exists(path_s):
    os.makedirs(path_s)
if not os.path.exists(path_lf):
    os.makedirs(path_lf)   
if not os.path.exists(path_chr):
    os.makedirs(path_chr)
if not os.path.exists(path_temp):
    os.makedirs(path_temp)
    
        
for fname in files:  
    bowtie_command = 'bowtie -v 2 -y -m 1 -a --best --strata -S -p 30 '\
        '--un %s%s --max %s%s --al %s%s %s %s%s %s%s 1>> %s 2>> %s'
        
        # first, align to ladder index to subtract any ladder oligos that were cloned        
    bowtie_ladder = bowtie_command %(
        path_l,
        fname+'_nomatch.fastq',
        path_l,
        fname+'_multi.fastq',
        path_l,
        fname+'_match.fastq',
        '^index/ladder/ladder',
        inpath,
        fname + bowtie_input_extension,
        path_temp,
        fname + '_ladder_match.SAM',
        log_file,
        log_file
        )
    ###
    logs = open(log_file, 'a+')
    logs.write('\n\n'+fname+'\n'+bowtie_ladder+'\n')
    logs.close()
    ###
    os.system(bowtie_ladder)        
        
        
        # second, align to tRNA index
    bowtie_tRNA = bowtie_command %(
        path_t,
        fname+'_nomatch.fastq',
        path_t,
        fname+'_multi.fastq',
        path_t,
        fname+'_match.fastq',
        '^index/MG1655/Ec_tRNA',
        path_l,
        fname +'_nomatch.fastq',
        path_temp,
        fname + '_tRNA_match.SAM',
        log_file,
        log_file
        )
    ###
    logs = open(log_file, 'a+')
    logs.write('\n'+bowtie_tRNA+'\n')
    logs.close()
    ###
    os.system(bowtie_tRNA)        
        
        
        # third, align to the rRNA index
    bowtie_rRNA = bowtie_command %(
        path_r,
        fname+'_nomatch.fastq',
        path_r,
        fname+'_multi.fastq',
        path_r,
        fname+'_match.fastq',
        '^index/MG1655/rRNA_MG1655',
        path_t,
        fname +'_nomatch.fastq',
        path_temp,
        fname + '_rRNA_match.SAM',
        log_file,
        log_file
        )
    ###
    logs = open(log_file, 'a+')
    logs.write('\n'+bowtie_rRNA+'\n')
    logs.close()
    ###
    os.system(bowtie_rRNA)         
        
        
        # forth, align to the ssrA_ssrS index
    bowtie_ssrAssrS = bowtie_command %(
        path_s,
        fname+'_nomatch.fastq',
        path_s,
        fname+'_multi.fastq',
        path_s,
        fname+'_match.fastq',
        '^index/ssrAssrS/Ec_ssrA_ssrS',
        path_r,
        fname +'_nomatch.fastq',
        path_temp,
        fname + '_ssrAssrS_match.SAM',
        log_file,
        log_file
        )
    ###
    logs = open(log_file, 'a+')
    logs.write('\n'+bowtie_ssrAssrS+'\n')
    logs.close()
    ###
    os.system(bowtie_ssrAssrS)
       
        
        # fifth, align to the lacI_ffs index
    bowtie_ssrAssrS = bowtie_command %(
        path_lf,
        fname+'_nomatch.fastq',
        path_lf,
        fname+'_multi.fastq',
        path_lf,
        fname+'_match.fastq',
        '^index/lacI_ffs/Ec_lacI_ffs',
        path_s,
        fname +'_nomatch.fastq',
        path_temp,
        fname + '_lacIfss_match.SAM',
        log_file,
        log_file
        )
    ###
    logs = open(log_file, 'a+')
    logs.write('\n'+bowtie_ssrAssrS+'\n')
    logs.close()
    ###
    os.system(bowtie_ssrAssrS)        

    
        # finally align to the chr index
    bowtie_chr = bowtie_command %(
        path_chr,
        fname+'_nomatch.fastq',
        path_chr,
        fname+'_multi.fastq',
        path_chr,
        fname+'_match.fastq',
        '^index/MG1655/e_coli_MG1655',
        path_lf,
        fname +'_nomatch.fastq',
        path_chr,
        fname + '_match.SAM',
        log_file,
        log_file
        )
    ###
    logs = open(log_file, 'a+')
    logs.write('\n'+bowtie_chr+'\n')
    logs.close()
    ###
    os.system(bowtie_chr)
    
logs = open(log_file, 'a+')
present = str(datetime.now())
logs.write('\n'+present+'\n')
logs.close()
    
print 
print 
for line in open(log_file, 'r'):
    print line[:-1]



2019-08-01 15:45:14.308918


ks175
bowtie -v 2 -y -m 1 -a --best --strata -S -p 30 --un ^resuts/alignments/ladder/ks175_nomatch.fastq --max ^resuts/alignments/ladder/ks175_multi.fastq --al ^resuts/alignments/ladder/ks175_match.fastq ^index/ladder/ladder ^fastq/ks175-unique-trimmed-tk.fastq ^resuts/tmpds/ks175_ladder_match.SAM 1>> ^resuts/log_bowtie.txt 2>> ^resuts/log_bowtie.txt
# reads processed: 23284030
# reads with at least one reported alignment: 194507 (0.84%)
# reads that failed to align: 22806818 (97.95%)
# reads with alignments suppressed due to -m: 282705 (1.21%)
Reported 194507 alignments to 1 output stream(s)

bowtie -v 2 -y -m 1 -a --best --strata -S -p 30 --un ^resuts/alignments/tRNA/ks175_nomatch.fastq --max ^resuts/alignments/tRNA/ks175_multi.fastq --al ^resuts/alignments/tRNA/ks175_match.fastq ^index/MG1655/Ec_tRNA ^resuts/alignments/ladder/ks175_nomatch.fastq ^resuts/tmpds/ks175_tRNA_match.SAM 1>> ^resuts/log_bowtie.txt 2>> ^resuts/log_bowtie.txt
# reads processed: 

In [8]:
def three_end_mapping(samgen,GFFgen_filename):
    from BCBio import GFF
    from Bio import Seq
    outputdata1={}
    outputdata2={}
    GFFgen=GFF.parse(GFFgen_filename)
    for sequence in GFFgen:
        outputdata1[sequence.id]=[0 for x in range(len(sequence))]
        outputdata2[sequence.id]=[0 for x in range(len(sequence))]
        for read in samgen:
            if read[0][0] == '@':   # Ignore header lines.
                continue
            if read[1] == '4':      # A bowtie mismatch. These are the same as those previously saved in the files bowtie output as reads without matches and reads exceeding the -m limit. No need to write as fastq.
                continue            
            chrom = read[2]             # chromosome identified for read in bowtie
            readid = read[0]            # read id
            startp = int(read[3]) - 1    # start position. Need to subtract 1 since genomic sequence starts at 1, but list structures for read summing start at 0.
            seq = Seq.Seq(read[9])      # sequence of the read
            length = len(seq)           # length of read
            if (read[1] == '16'):
                start = startp
                outputdata2[chrom][start] += 1
            if (read[1] == '0'):
                start = startp + length - 1
                outputdata1[chrom][start] += 1
    return [outputdata1,outputdata2]  

def readcounts_to_rpm(readcounts):
    for chrom in readcounts.keys():
        totalread = sum(readcounts[chrom])
        for position in range(len(readcounts[chrom])):
            readcounts[chrom][position]/=float(totalread)
            readcounts[chrom][position]*=1E6

# makes density file 
# the last letter "f" in writecountsf means float
def writecountsf(rpm,filepath):  #Resultlist it is the list of counts, filestring is the file prefix for each chr to be written.
    import struct
    f2=open(filepath+"keys","w")
    for chrom in rpm.keys():
        f=open(filepath+"genome","wb")
        # used to be (filepath+chrom,"wb") but server fail to show a file with a name containing "|"
        # rename "genome" to real genome name manually 
        for position in rpm[chrom]:
            f.write(struct.pack("f",position))
            # struct.pack to make binary based on the format indicated 
            # here, the indication of format is "f", which means float
        f.close()
        f2.write(chrom+"\n")
    f2.close()

# makes wig file
def countstowig(rpm,filestring):
    import random
    f=open(filestring+".wig","w")
    filestring=filestring.partition("_")[0][-3:]

    random.seed(filestring)
    c1=random.randint(0,255)
    random.seed(filestring+"xxx")
    c2=random.randint(0,255)
    random.seed(filestring+"000")
    c3=random.randint(0,255)

    f.write("track name=tracklabel viewLimits=-5:5 color="+str(c1)+','+str(c2)+','+str(c3)+"\n")
    for chrom in rpm.keys():
        if chrom[0:3]=='chr':
            f.write("fixedStep  chrom="+chrom+"  start=1  step=1\n")
        else:
            f.write("fixedStep  chrom=\""+chrom+"\"  start=1  step=1\n")	# Note the extra '' around chrom added for E coli. Should be fine for yeast. Actually it's not. So new code here to detect that.


        for position in rpm[chrom]:
            f.write(str(position)+"\n")
    f.close()

In [9]:
# 3-end mapping for denisty

######################################################
import csv
import pickle
from copy import deepcopy

out_path=filepath+'density/'
if not os.path.exists(out_path):
    os.makedirs(out_path)

for f in files:
    samgenmain_filename=filepath+'alignments/chr/'+f+'_match.SAM'
    f_samgenmain=open(samgenmain_filename)
    samgenmain=csv.reader(f_samgenmain,delimiter='\t')

    gff = 'coli.gff'

    three_mapped=three_end_mapping(samgenmain,gff)
    pickle_readcounts = out_path+f+mapping+'_readcounts.pickle'
    with open(pickle_readcounts, 'wb') as readcounts_f:
        pickle.dump(three_mapped,readcounts_f)

    three_mapped_rpm = deepcopy(three_mapped)
    readcounts_to_rpm(three_mapped_rpm[0])
    readcounts_to_rpm(three_mapped_rpm[1])
    pickle_rpm = out_path+f+mapping+'_rpm.pickle'
    with open(pickle_rpm, 'wb') as rpm_f:
        pickle.dump(three_mapped_rpm,rpm_f)
        
    density_path = out_path+f+mapping
    writecountsf(three_mapped_rpm[0],density_path+"_plus_")
    countstowig(three_mapped_rpm[0],density_path+"_plus")
    writecountsf(three_mapped_rpm[1],density_path+"_minus_")
    countstowig(three_mapped_rpm[1],density_path+"_minus")

In [10]:
# RPKM

in_path = filepath+'density/'
out_path = filepath+'expression/'
if not os.path.exists(out_path):
    os.makedirs(out_path)

#########################################################################################

with open('Ecoli_Gene_TU.pickle','rb') as GT_f:
    Gene_TU_dict = pickle.load(GT_f)

for f in files:
    expression_dict={}
    out_pickle=out_path+f+mapping+gene_type+'_expression.pickle'
    out_csv=out_path+f+mapping+gene_type+'_expression.csv'
    readcounts_file = in_path+f+mapping+'_readcounts.pickle'
    
    with open(readcounts_file, 'rb') as rf:
        readcounts_dict = pickle.load(rf)
    plus_readcounts = readcounts_dict[0][readcounts_dict[0].keys()[0]]
    minus_readcounts = readcounts_dict[1][readcounts_dict[1].keys()[0]]
    total_read = sum(plus_readcounts)+sum(minus_readcounts)
    
    for key in Gene_TU_dict.keys():
        if key in expression_dict.keys():
            print key +' is overlapping!!!'
            continue
        start = int(Gene_TU_dict[key][2])-1
        end = int(Gene_TU_dict[key][3])
        length = end-start
        if length < 0:
            print key + 'is shorter than zero!!!'
            continue
        if Gene_TU_dict[key][0] == '+':
            reads = float(sum(plus_readcounts[start:end]))
        if Gene_TU_dict[key][0] == '-':
            reads = float(sum(minus_readcounts[start:end]))
        rpc = (reads/length)*3
        rpkm = (((reads/total_read)*1000000)/length)*1000
        expression_dict[key]=[]
        expression_dict[key].append(reads)
        expression_dict[key].append(rpc)
        expression_dict[key].append(rpkm)
    
    with open(out_pickle, 'wb') as op:
        pickle.dump(expression_dict,op)           
    
    rpkm_DataFrame = DataFrame.from_dict(expression_dict, orient='index')
    rpkm_DataFrame.columns = ['reads','rpc','rpkm']
    rpkm_DataFrame.to_csv(out_csv) 